# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [11]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [20]:
data_agg = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'})

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [15]:
data_pivot = pd.pivot_table(data, index=['ProductName'], 
                            values = ['Quantity'], columns = ['CustomerID']).fillna(0)

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [17]:
from scipy.spatial.distance import pdist, squareform

matrix = squareform(pdist(data_pivot.T, 'euclidean'))

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [39]:
distances = pd.DataFrame(1/ (1 + matrix),
                         index = data_pivot.columns, columns = data_pivot.columns)

top_five_similarities = distances['Quantity'][33].sort_values(ascending= False)[1:6] # using cust ID 33

top_five_similarities

          CustomerID
Quantity  3909          0.095358
          264           0.093953
          3531          0.093953
          2503          0.093498
          3305          0.093051
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [55]:
customers = top_five_similarities.unstack().columns.values

In [56]:
data_agg.iloc[customers]

,,Quantity
CustomerID,ProductName,
5434,Skirt - 29 Foot,2
412,Cheese - Camembert,1
4723,Cinnamon Buns Sticky,2
2939,Bread - Italian Corn Meal Poly,1
4261,Soup Knorr Chili With Beans,2


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [63]:
quant = data[data['CustomerID'].isin(customers)].groupby('ProductName').agg({'Quantity':'sum'}).sort_values(by='Quantity', ascending =False)

In [64]:
quant.head()

,Quantity
ProductName,
"Salsify, Organic",4
Quiche Assorted,3
Chocolate - Dark,3
Wine - Charddonnay Errazuriz,3
Bay Leaf,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [71]:
products_purchased = data[data['CustomerID'] == 33].groupby('ProductName').agg({'Quantity':'sum'}).T.columns.values

In [96]:
not_purchased = [product for product in quant.index.values if product not in products_purchased]

In [112]:
temp = quant.reset_index()
recommendations = temp[temp['ProductName'].isin(not_purchased)].head(5).reset_index(drop=True)
recommendations

,ProductName,Quantity
0,Chocolate - Dark,3
1,Wine - Charddonnay Errazuriz,3
2,Bay Leaf,3
3,"Oranges - Navel, 72",3
4,Wine - Ej Gallo Sierra Valley,3


In [115]:
print('Recommendations:\n', *recommendations['ProductName'].values, sep='\n')

Recommendations:

Chocolate - Dark
Wine - Charddonnay Errazuriz
Bay Leaf
Oranges - Navel, 72
Wine - Ej Gallo Sierra Valley


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [124]:
custIds = data['CustomerID'].unique()
cust_dict = {}

for custId in custIds:
    top_five_similarities = distances['Quantity'][custId].sort_values(ascending= False)[1:6]
    customers = top_five_similarities.unstack().columns.values
    records =  data_agg.iloc[customers]
    quant = data[data['CustomerID'].isin(customers)].groupby('ProductName').agg({'Quantity':'sum'}).sort_values(by='Quantity', ascending =False)
    products_purchased = data[data['CustomerID'] == 33].groupby('ProductName').agg({'Quantity':'sum'}).T.columns.values
    not_purchased = [product for product in quant.index.values if product not in products_purchased]
    temp = quant.reset_index()
    recommendations = temp[temp['ProductName'].isin(not_purchased)].head(5).reset_index(drop=True)
    cust_dict[custId] = recommendations['ProductName'].values

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [138]:
columns = ['CustomerID', 'P1', 'P2', 'P3', 'P4', 'P5'] # where P represents 'product'
temp = pd.DataFrame(cust_dict).T.reset_index()
temp.columns = columns
temp.head()

,CustomerID,P1,P2,P3,P4,P5
0,61288,"Mushrooms - Black, Dried","Wine - Magnotta, Merlot Sr Vqa",Sponge Cake Mix - Chocolate,Beef - Rib Eye Aaa,Chicken - Soup Base
1,77352,Guinea Fowl,Grenadine,Ecolab - Mikroklene 4/4 L,"Oranges - Navel, 72","Coconut - Shredded, Sweet"
2,40094,"Water - Mineral, Natural",Pasta - Orecchiette,"Oregano - Dry, Rubbed",Wiberg Super Cure,Tea - Decaf Lipton
3,23548,Wanton Wrap,Spinach - Baby,Tea - English Breakfast,Banana Turning,Bread - Raisin Walnut Oval
4,78981,Lettuce - Frisee,Longos - Chicken Wings,Pop Shoppe Cream Soda,Crush - Cream Soda,Sprouts - Alfalfa


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [148]:
matrix = squareform(pdist(data_pivot.T, 'cityblock'))

distances = pd.DataFrame(1/ (1 + matrix),
                         index = data_pivot.columns, columns = data_pivot.columns)

In [149]:
cust_dict = {}

for custId in custIds:
    top_five_similarities = distances['Quantity'][custId].sort_values(ascending= False)[1:6]
    customers = top_five_similarities.unstack().columns.values
    records =  data_agg.iloc[customers]
    quant = data[data['CustomerID'].isin(customers)].groupby('ProductName').agg({'Quantity':'sum'}).sort_values(by='Quantity', ascending =False)
    products_purchased = data[data['CustomerID'] == 33].groupby('ProductName').agg({'Quantity':'sum'}).T.columns.values
    not_purchased = [product for product in quant.index.values if product not in products_purchased]
    temp = quant.reset_index()
    recommendations = temp[temp['ProductName'].isin(not_purchased)].head(5).reset_index(drop=True)
    cust_dict[custId] = recommendations['ProductName'].values

In [150]:
columns = ['CustomerID', 'P1', 'P2', 'P3', 'P4', 'P5'] # where P represents 'product'
temp = pd.DataFrame(cust_dict).T.reset_index()
temp.columns = columns
temp.head()

,CustomerID,P1,P2,P3,P4,P5
0,61288,Pork - Kidney,"Cheese - Boursin, Garlic / Herbs",Wine - Ej Gallo Sierra Valley,Butter - Unsalted,Bay Leaf
1,77352,Cheese - Camembert,Beef - Montreal Smoked Brisket,Pasta - Angel Hair,Sausage - Liver,Pomello
2,40094,Wine - Ej Gallo Sierra Valley,Sugar - Fine,Crush - Cream Soda,Tia Maria,Tea - Decaf Lipton
3,23548,Muffin - Carrot Individual Wrap,Pork - Kidney,Soup - Campbells Tomato Ravioli,Beef - Texas Style Burger,Lamb - Ground
4,78981,Watercress,Knife Plastic - White,Butter - Unsalted,Smirnoff Green Apple Twist,Beef - Montreal Smoked Brisket
